In [1]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import math
from sklearn.model_selection import train_test_split
from functools import reduce
from functools import partial
import operator
from timeit import default_timer
from matplotlib.ticker import FormatStrFormatter
import deepxde as dde
import time

Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.


In [2]:
def set_size(width, fraction=1, subplots=(1, 1), height_add=0):
    """Set figure dimensions to avoid scaling in LaTeX.
    ###设置图形尺寸，避免在 LaTeX 中缩放

    Parameters
    ----------
    width: float or string
            Document width in points, or string of predined document type
            ###以点为单位的文档宽度，或预定文档类型的字符串
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy
            ###您希望图形所占宽度的分数
    subplots: array-like, optional
            The number of rows and columns of subplots.
            ###子绘图的行数和列数。
    Returns
    -------
    fig_dim: tuple
            ### The primary difference is that we cannot modify a tuple once it is created.
            Dimensions of figure in inches
    """
    if width == 'thesis':
        width_pt = 426.79135
    elif width == 'beamer':
        width_pt = 307.28987
    else:
        width_pt = width
    ### 确定文档的类型从而选择生成图片区域的大小

    # Width of figure (in pts)
    # 图标在区域中占的宽度，并转换为英寸的单位
    fig_width_pt = width_pt * fraction
    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5**.5 - 1) / 2
    # 0.618

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = height_add + fig_width_in * golden_ratio * (subplots[0] / subplots[1])

    return (fig_width_in, fig_height_in)

tex_fonts = {
    # Use LaTeX to write all text
    "text.usetex": True,
    "font.family": "times",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 10,
    "font.size": 10,
    # Make the legend/bel fonts a little smaller
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
}

plt.rcParams.update(tex_fonts)
# plt.rcParams.update()，其中 update 是 rcParams 对象的一个方法，用于更新 Matplotlib 的运行时配置参数。


In [3]:
def solvePDEAdaptive(I, a, L, dt,F, T, lam, solveControl, kernelEstimator, gamma, lamGuess, grid, model, debug):
    # Code adapted from: https://hplgit.github.io/fdm-book/doc/pub/book/pdf/fdm-book-4print.pdf
    Nt = int(round(T/float(dt)))
    t = np.linspace(0, Nt*dt, Nt)  
    dx = np.sqrt(a*dt/F)
    Nx = int(round(L/dx))
    x = np.linspace(0, L, Nx+1)      
    # Make sure dx and dt are compatible with x and t
    dx = x[1] - x[0]
    dt = t[1] - t[0]
    lArr = []
    kArr = []
    lHat = lamGuess
    lArr.append(lHat)

    u = np.zeros((Nt, Nx+1))

    # Set initial condition u(x,0) = I(x)
    for i in range(0, Nx+1):
        u[0][i] = I[i]
    s= 0
    for i in range(1, Nt):
        kEst = kernelEstimator(lHat, dx, model, grid)
        if i % int(Nt/100) == 0 and debug==True:
            print("i", i, "/", nt)
            print("wTerm", wTerm)
            print("uTerm", uTerm)
            print("lhatdelta", lHatDelta)
            print(time.time() - s)
            s = time.time()
        kArr.append(kEst)
            
        # Compute u at inner mesh points
        u[i][1:Nx] = u[i-1][1:Nx] +  \
                      F*(u[i-1][0:Nx-1] - 2*u[i-1][1:Nx] + u[i-1][2:Nx+1]) + dt*lam[1:Nx]*u[i-1][1:Nx]

        lHatDelta,wTerm, uTerm = estimateLambdaHatDelta(x, u[i-1], kEst, dx, gamma)
        lHat = lHat + dt*lHatDelta
        # Insert boundary conditions
        u[i][0] = 0;  u[i][Nx] = solveControl(u[i], kEst, Nx-1, dx)
        lArr.append(lHat)
    return u, x, lArr, kArr

def solvePDE(I, a, L, dt,F, T, lam, solveControl, kernel):
    # Code adapted from: https://hplgit.github.io/fdm-book/doc/pub/book/pdf/fdm-book-4print.pdf  
    Nt = int(round(T/float(dt)))
    # round()如果只有一个数作为参数，不指定位数的时候，返回的是一个整数，而且是最靠近的整数（这点上类似四舍五入）。但是当出现.5的时候，两边的距离都一样，round()取靠近的偶数。
    # 当指定取舍的小数点位数的时候：一般情况也是使用四舍五入的规则，但是碰到.5的这样情况，如果要取舍的位数前的小数是奇数，则直接舍弃，如果偶数这向上取舍。
    t = np.linspace(0, Nt*dt, Nt)  
    dx = np.sqrt(a*dt/F)
    Nx = int(round(L/dx))
    x = np.linspace(0, L, Nx+1)      
    # Make sure dx and dt are compatible with x and t
    dx = x[1] - x[0]
    dt = t[1] - t[0]

    u = np.zeros((Nt, Nx+1))

    # Set initial condition u(x,0) = I(x)
    # 0时刻不同位置的初始值
    # range(a,b)不包括b
    for i in range(0, Nx+1):
        u[0][i] = I[i]

    for i in range(1, Nt):
        if i % int(Nt/10) == 0:
            print("i", i, "/", Nt)
        # Compute u at inner mesh points
        # 在内部网格点计算 u
        u[i][1:Nx] = u[i-1][1:Nx] +  \
                      F*(u[i-1][0:Nx-1] - 2*u[i-1][1:Nx] + u[i-1][2:Nx+1]) + dt*lam[1:Nx]*u[i-1][1:Nx]

        # Insert boundary conditions
        # 插入边界条件
        # 左边界条件即 x = 0 ; 右边界条件需要根据当前时刻的解与kernel来求得
        u[i][0] = 0;  
        u[i][Nx] = solveControl(u[i], kernel, Nx-1, dx)
    return u

def solveKernelFunction(lam, dx, _a, _b):
    k = np.zeros((len(lam), len(lam)))
    # First we calculate a at each timestep
    a = lam

    # FD LOOP
    k[1][1] = -(a[1] + a[0]) * dx / 4
    for i in range(1, len(lam)-1):
        k[i+1][0] = 0
        k[i+1][i+1] = k[i][i]-dx/4.0*(a[i-1] + a[i])
        k[i+1][i] = k[i][i] - dx/2 * a[i]
        for j in range(1, i):
                k[i+1][j] = -k[i-1][j] + k[i][j+1] + k[i][j-1] + a[j]*(dx**2)*(k[i][j+1]+k[i][j-1])/2
    return k

def solveKernelFunctionNOP(lam, dx, model, grid):
    torchLam = torch.from_numpy(lam).reshape((1, len(lam))).cuda().float()
    out = model((torchLam, grid))
    out = out.cpu().detach().numpy()
    out = np.reshape(out, (len(lam), len(lam)))
    return out

def solveLambdaFunction(x, gamma):
    lam = np.zeros(nx)
    for idx, val in enumerate(x):
        lam[idx] = 5
    return lam

# solveControl 函数将一个核函数 kernel 应用于解向量 u，通过加权求和和数值积分的方式，计算并返回某个边界条件或控制值。这个值可能在求解 PDE 时用于设置边界条件，以影响后续的计算过程。
def solveControl(u, kernel, nx, dx):
    return sum(kernel[-1][0:nx+1]*u[0:nx+1])*dx
# 取核函数的最后一行与模拟求解得到的u（这里每个时刻计算一次所以传入的实际为u[i],两者取0到Nx-1项，逐项相乘再求和，最后乘以dx得到u[i][Nx]）


def openLoop(u, kernel, nx, dx):
    return 0

def zeroToNan(x):
    for i in range(len(x)):
        for j in range(len(x[0])):
            if j >= i:
                x[i][j] = float('nan')
    return x

# PDE L2 Error
def getPDEl2(u, uhat):
    nt = len(u)
    nx = len(u[0])
    pdeError = np.zeros(nt-1)
    for i in range(1, nt):
        error = 0
        for j in range(nx):
            error += (u[i][j] - uhat[i][j])**2
        error = np.sqrt(error*0.01)
        pdeError[i-1] = error
    return pdeError

def solveW(u, kernel, dx):
    w = np.zeros(len(kernel))
    for i in range(0, len(kernel)):
        w[i] = u[i]-sum(kernel[i][0:i]*u[0:i])*dx
    return w

def solveWIntegral(x, w, dx):
    return sum(w[0:]**2)*dx

def estimateLambdaHatDelta(x, u, k, dx, gamma):
    w = solveW(u, k, dx)
    wInt = solveWIntegral(x, w, dx)
    lambdaHat = np.zeros(len(w))
    kt= k.transpose()
    for i in range(0, len(w)):
        middleTerm = (w[i] - sum(kt[i][i:]*w[i:])*dx)*u[i]
        lambdaHat[i] = gamma/(1+wInt)*middleTerm 
    return lambdaHat, wInt, middleTerm

def nanToZero(x):
    for i in range(len(x)):
        for j in range(len(x[0])):
            if j >= i:
                x[i][j] = float('nan')
    return x

def removeTriangle(k):
    newK = k.copy()
    for i in range(len(k)):
        for j in range(len(k)):
            if j>i:
                newK[i][j] = np.nan
    return newK

In [4]:
X = 1
dx = 0.01
nx = int(round(X/dx))
spatial = np.linspace(0, X, nx+1, dtype=np.float32)
print(nx)



T = .5
dt = 0.00001
nt = int(round(T/dt))
temporal = np.linspace(0, T, nt, dtype=np.float32)
print(nt)
# spatial

100
50000


In [ ]:
# Dataset generation. Keep init_cond, and the guess fixed. 
# 生成数据集。保持 init_cond 和猜测值不变。
init_cond = np.zeros(nx+1)
init_cond[:] = 20
lamGuess = np.ones(nx+1)*25
lam = np.zeros(nx+1)
start = time.time()
# Sample every 100 timesteps. Subtract 2 bc kArr only has 49999 elements. 
idxs = np.linspace(0, nt-2, int(nt/100), dtype=np.int64)
# 每隔一百个点设置一个索引
x = []
y = []
for i in range(10):
    gamma = np.random.uniform(8.5, 9.5)
    for j in range(len(spatial)):
        lam[j] = 25*math.cos(gamma*math.acos(spatial[j])) +25
    # _a, _b, lArr, kArr= solvePDEAdaptive(init_cond, 1, 1, dt, dt/dx**2, T, lam, solveControl, solveKernelFunction, 50, lamGuess, lamGuess, lamGuess, False)
    _a, _b, lArr, kArr= solvePDEAdaptive(init_cond, 1, 1, dt, dt/dx**2, T, lam, solveControl, solveKernelFunction, gamma, lamGuess, lamGuess, lamGuess, False)
    ## 50替换为gamma？ 第二个和第三个lamGuess是否有错误？（没有错误，因为保证参数和类似函数输入一致性而设置的，后两个lanGuess可以任意设置）
    for val in idxs:
        x.append(lArr[val])
        y.append(kArr[val])
x = np.array(x)
y = np.array(y)
print(x.shape)
print(y.shape)
print("Total Dataset Generation Time:", time.time()-start)
arr_reshaped = y.reshape(y.shape[0], -1)
# numpy 数组的 shape 属性返回数组的维数。y.shape = (m,n), y.shape[0]表示y的行数 ，-1表示自适应调整列数
np.savetxt("y.dat", arr_reshaped)
np.savetxt("x.dat", x)